In [1]:
from dataInput import *
from CV import *
from TwoStageSP import *
from FA import *
import numpy as np
import pandas as pd

In [2]:
inputParams = inputParams(1,1,57,1000);
solveParams = solveParams(100000,200,100000,10800,1e-4);

In [3]:
intensityFile = 'data/synthetic/intensity.csv';
locationFile = 'data/synthetic/location.csv';
landfallFile = 'data/synthetic/landfall_7.csv';

In [4]:
hurricaneDataSet = hurricaneInput(intensityFile, locationFile, landfallFile, inputParams)

In [5]:
netNodesFile = 'data/synthetic/nodes.csv';
netParamsFile = 'data/synthetic/netParams.csv';
networkDataSet = networkInput(3,10,0.5,netNodesFile,netParamsFile,hurricaneDataSet)

In [6]:
osfname = "./data/synthetic/OOS" + str(inputParams.k_init) + ".csv"
OS_paths = pd.read_csv(osfname).values  # Read the out-of-sample file

In [7]:
t_roll = 0;
x_init = networkDataSet.x_0;
s = 1;
k_t = inputParams.k_init-1

In [8]:
master, x, f, theta, subproblem, y2, xCons, dCons, rCons = RH_2SSP_define_models(networkDataSet, hurricaneDataSet, inputParams, t_roll, k_t, x_init)

Set parameter Username
Academic license - for non-commercial use only - expires 2024-07-30


In [9]:
LB, UB, xval, fval, thetaval = RH_2SSP_solve_roll(networkDataSet, hurricaneDataSet, inputParams, solveParams, k_t, t_roll, master, subproblem, x, f, theta, y2, xCons, dCons, rCons)

TypeError: unsupported operand type(s) for *: 'float' and 'generator'